## Introduction

Once data is stored as a dataframe, there are many nuts-and-bolts operations to add new columns, drop columns, resample data, aggregate data, join multiple tables, concatenate rows, and the like.  

We use the [pandas](https://pandas.pydata.org/) framework for handling data, chiefly because it has a lot of built-in functions for handling data like ours, which is to say time-indexed multivariate data.



## Working with indices

One of the main advantages of using a framework like pandas is the ability to index the data, which essentially formalizes your intuition that data is organized as a timeseries with some number of columns.  With that index on time, you can join this table with other tables indexed on time, _even if they have different timestamps_.  Pandas has useful functions to resample to a consistent time interval (for example to go from 3-hourly to 1-hourly or 1-hourly to daily), and along with this come rules for what to do during that resampling or aggregating.  

A few applications we'll look at here are

* Setting the index to local time using the time offset from `location`.  You may use this when comparing Mark data to some local data source.

* Calculating daily maximum, minimum, average, and other such metrics from hourly data.

* Joining hourly data (such as reference evapotranspiration or ET) with daily data (such as a crop coefficient) to calculate crop ET.  

First things first let's import the modules we need and download a data frame:

In [10]:
import pandas as pd
from datetime import date, datetime, timedelta
from io import StringIO
from arable.client import ArableClient
import os
import requests

In [5]:
arable_email = os.getenv('ARABLE_EMAIL')
arable_passwd = os.getenv('ARABLE_PASSWD')
arable_tenant = os.getenv('ARABLE_TENANT')

a = ArableClient()
a.connect(arable_email, arable_passwd, arable_tenant)

device = 'A000176' 

sta = "2018-07-04 08:00:00"
end = "2018-07-07 08:00:00"

hourly = a.query(select='all', 
             format='csv', 
             devices=[device], 
             measure='hourly', 
             order='time', 
             end=end, start=sta) 

hourly = StringIO(hourly)
hourly = pd.read_csv(hourly, sep=',', error_bad_lines=False)

Next order of business is to set the `time` column as datetime object as python understands them.  

In [7]:
hourly['time'] = pd.to_datetime(hourly['time'])
hourly.index = hourly['time']

If we look at this dataframe we now see that it is indexed.  If we do anything to this dataframe, like create a new column (perhaps as a calculation from existing columns) it will preserve the integrity of the timeseries.

In [8]:
hourly

,time,device,location,lat,long,B1dw,B1uw,B2dw,B2uw,B3dw,...,PARuw,RH,SWdw,SWuw,Tabove,Tair,Tbelow,Tdew,prate,precip
time,,,,,,,,,,,,,,,,,,,,,
2018-07-04 08:00:00,2018-07-04 08:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,0.051308,0.067110,0.038234,0.045859,0.061278,...,-3.57414,94.788760,1.26640,-3.99047,11.58240,20.7853,20.0060,19.8947,NaN,NaN
2018-07-04 09:00:00,2018-07-04 09:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,0.056608,0.066005,0.040373,0.047362,0.062360,...,-3.56883,89.374411,1.27809,-3.98454,12.64880,22.3720,21.3450,20.5205,NaN,NaN
2018-07-04 10:00:00,2018-07-04 10:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,0.054730,0.067325,0.039561,0.046097,0.061172,...,-3.44494,97.148591,1.28095,-3.84622,11.82400,20.9027,20.4938,20.4081,NaN,NaN
2018-07-04 11:00:00,2018-07-04 11:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,0.162753,0.064567,0.266783,0.047117,0.278131,...,-3.60956,100.000000,7.14933,-4.03001,11.22020,19.6604,19.4976,19.6467,NaN,NaN
2018-07-04 12:00:00,2018-07-04 12:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,1.162560,0.060473,1.951420,0.129333,1.984490,...,-8.38327,100.000000,52.58780,-9.35980,11.56200,20.2664,20.3475,20.2527,NaN,NaN
2018-07-04 13:00:00,2018-07-04 13:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,3.444750,0.053626,6.544960,0.432657,6.840740,...,-26.12080,97.132850,181.73800,-29.16350,12.94210,22.2324,22.6574,21.7390,NaN,NaN
2018-07-04 14:00:00,2018-07-04 14:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,6.038870,0.053297,12.292000,0.621580,13.042900,...,-36.00820,91.699177,336.70000,-40.20260,14.55970,24.4235,25.6542,22.9652,NaN,NaN
2018-07-04 15:00:00,2018-07-04 15:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,9.013480,0.061583,18.391800,0.793171,19.423400,...,-46.25820,84.239173,492.95300,-51.64650,14.13640,25.1399,26.9049,22.2687,NaN,NaN
2018-07-04 16:00:00,2018-07-04 16:00:00,A000176,5b043a2f17edbb00015afa42,41.0876,-100.774,12.566300,0.072604,25.303800,1.003910,26.226900,...,-59.19120,74.012285,667.21200,-66.08600,15.26440,28.1742,30.3608,23.0552,NaN,NaN


Our time is stored as UTC, where the "U" stands for Universal.  It is a format that is easy for machines.  But humans tend to prefer local time. We saw in [example 1](https://pro-soap.cloudvent.net/ex1_Downloading.html) that it was possible to grab the local time offset from the `locations` data structure associated with the device

In [11]:
auth_token = a.header['Authorization']
location_id = a.devices(name='A000176')['location']['id']

base = 'https://api-user.arable.cloud/api/v1/'
path = '/locations/'
url = base + path + location_id

response = requests.get(url, headers = {'Authorization': auth_token})
location = response.json()

time_offset = location['time_offset']

We use `timedelta` to apply the time offset to the UTC time, adding a column using `['column_name']` syntax.  We then set it as our index.

In [12]:
hourly['local_time'] = hourly['time'] + timedelta(seconds=time_offset)
hourly.index = hourly['local_time']

Now, any aggregation or resampling (such as daily max and min) will be performed using local time instead of UTC time.

## Retaining columns

In [23]:
health = a.query(select='all', 
             format='csv', 
             devices=[device], 
             measure='health', 
             order='time', 
             end=end, start=sta) 

health = StringIO(health)
health = pd.read_csv(health, sep=',', error_bad_lines=False)
health['time'] = pd.to_datetime(health['time'])


In [24]:
health = health[['time','rssi','batt_volt']]

## Dealing with NAs

In [25]:
health = health.fillna(method='bfill')

## Resampling

In [28]:
health = health.resample('H').mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [ ]:
## Inner join

One of the useful features of using indexed dataframes is the ease of joining tables.  The join always happens using a unique identifier (UID) that both tables have in common.  For us, the obvious UID is the time index.  

In database world there are two basic joins: the "inner join" and the "outer join".  And within outer joins there are "left outer joins" and "right outer joins".  To explain, let's imagine two tables (A and B) with the indices shown:

```
A B
1 1
2 
3 3
  4
5 5
```

An _inner join_ returns only the rows where the index is present in both tables:

```
AB
1
3
5
```

A _left outer join_ returns all the rows where the left table has an index, and inserts blank values where the right table has nothing:

```
AB
1
2
3
5
```

Similarly for a right outer join:

```
AB
1
3
4
5
```

One common join for us is to join the `health` table to the `hourly` table:

In [22]:
health

,time,rssi,batt_volt
0,2018-07-04 08:02:37,-67.0,4102.0
1,2018-07-04 08:07:37,-67.0,4102.0
2,2018-07-04 08:12:38,-67.0,4102.0
3,2018-07-04 08:17:39,-67.0,4102.0
4,2018-07-04 08:22:39,-67.0,4102.0
5,2018-07-04 08:27:40,-67.0,4102.0
6,2018-07-04 08:32:41,-67.0,4101.0
7,2018-07-04 08:37:41,-67.0,4101.0
8,2018-07-04 08:42:42,-67.0,4090.0
9,2018-07-04 08:47:43,-73.0,4098.0


In [ ]:
health['local_time'] = health['time'] + timedelta(seconds=time_offset)
health.index = health['local_time']

In [ ]:
And now the join is accomplished with the call to `merge`.

In [ ]:
df = pd.merge(left=hourly,right=health,on='local_time')


## Filtering

```
mask = (sdf['solartime'] >= start) 
idf = sdf.loc[mask].copy()
```

## Dropping columns

## Outer join, forward fill (e.g. for a custom kc)

```
tdf = tdf.index.normalize()
tdf = tdf.merge(sdf, how='outer', sort=True)
tdf = tdf.fillna(method='ffill')
```

## Diurnal temperature range


## Groupby, resample, sum to get daily or weekly metrics
https://stackoverflow.com/questions/45281297/group-by-week-in-pandas

```
df = df.resample('30S').ffill().resample('5T').mean()
```

```
tdf = tdf.groupby(pd.Grouper(freq='D')).min()
```

## Concatenate multiple devices - weekly report
